In [0]:
from pyspark.sql import DataFrameWriter
import pandas as pd
from pyspark.sql.functions import *

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='scope31dec')]

In [0]:
dbutils.secrets.list(scope = 'scope31dec')

[SecretMetadata(key='blobkeysecret31dec')]

In [0]:
spark.conf.set("fs.azure.account.key.healthcareblob31dec.dfs.core.windows.net",
    dbutils.secrets.get(scope="scope31dec", key="blobkeysecret31dec"))

In [0]:
display(dbutils.fs.ls("abfss://rawdata@healthcareblob31dec.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/Patient_records.csv,Patient_records.csv,5110,1704010961000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/disease.csv,disease.csv,1489,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/group.csv,group.csv,4390,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/hospital.csv,hospital.csv,1328,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/subgroup.csv,subgroup.csv,561,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/subscriber.csv,subscriber.csv,12061,1704010960000


In [0]:
group_data = spark.read.csv("abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/group.csv", header = True, inferSchema= True)

In [0]:
group_data.show(10, False)

+-------+---------------+--------+------+----------------------------------------+--------+---------+
|country|premium_written|zip_code|grp_id|grp_name                                |grp_type|city     |
+-------+---------------+--------+------+----------------------------------------+--------+---------+
|India  |72000          |482018  |GRP101|Life Insurance Corporation of India     |Govt.   |Mumbai   |
|India  |45000          |482049  |GRP102|HDFC Standard Life Insurance Co. Ltd.   |Private |Mumbai   |
|India  |64000          |482030  |GRP103|Max Life Insurance Co. Ltd.             |Private |Delhi    |
|India  |59000          |482028  |GRP104|ICICI Prudential Life Insurance Co. Ltd.|Private |Mumbai   |
|India  |37000          |482014  |GRP105|Kotak Mahindra Life Insurance Co. Ltd.  |Private |Mumbai   |
|India  |89000          |482011  |GRP106|Aditya Birla Sun Life Insurance Co. Ltd.|Private |Mumbai   |
|India  |70000          |482006  |GRP107|TATA AIG Life Insurance Co. Ltd.        |

In [0]:
group_data.printSchema()

root
 |-- country: string (nullable = true)
 |-- premium_written: integer (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- grp_id: string (nullable = true)
 |-- grp_name: string (nullable = true)
 |-- grp_type: string (nullable = true)
 |-- city: string (nullable = true)



In [0]:
#group_data.filter(col('group_data') in ["GRP110", "GRP130", "GRP156"]).show(5,False)

---------------------------------------------------------------------------
PySparkValueError                         Traceback (most recent call last)
File <command-2178660243309083>, line 1
----> 1 group_data.filter(col('group_data') in ["GRP110", "GRP130", "GRP156"]).show(5,False)

File /databricks/spark/python/pyspark/sql/column.py:1514, in Column.__nonzero__(self)
   1513 def __nonzero__(self) -> None:
-> 1514     raise PySparkValueError(
   1515         error_class="CANNOT_CONVERT_COLUMN_INTO_BOOL",
   1516         message_parameters={},
   1517     )

PySparkValueError: [CANNOT_CONVERT_COLUMN_INTO_BOOL] Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [0]:
group_data.select("*").filter(col("grp_id").isin("GRP110", "GRP130", "GRP156")).show(5,False)

+-------+---------------+--------+------+-------------------------------------+--------+---------+
|country|premium_written|zip_code|grp_id|grp_name                             |grp_type|city     |
+-------+---------------+--------+------+-------------------------------------+--------+---------+
|India  |48000          |482015  |GRP110|Bajaj Allianz Life Insurance Co. Ltd.|Private |Pune     |
|India  |42000          |482036  |GRP130|Bharti AXA General Insurance         |Private |Mumbai   |
|India  |67000          |482048  |GRP156|The Oriental Insurance Company       |Govt.   |New Delhi|
+-------+---------------+--------+------+-------------------------------------+--------+---------+



In [0]:
#check duplicates
group_data.groupby(group_data.columns).count().where("count >1").show()

+-------+---------------+--------+------+--------+--------+----+-----+
|country|premium_written|zip_code|grp_id|grp_name|grp_type|city|count|
+-------+---------------+--------+------+--------+--------+----+-----+
+-------+---------------+--------+------+--------+--------+----+-----+



In [0]:
group_data = group_data.drop_duplicates()

In [0]:
group_data.select([count(when(isnan(c) | col(c).isNull(), c)). alias(c) for c in group_data.columns]).show()

+-------+---------------+--------+------+--------+--------+----+
|country|premium_written|zip_code|grp_id|grp_name|grp_type|city|
+-------+---------------+--------+------+--------+--------+----+
|      0|              0|       0|     0|       0|       0|   0|
+-------+---------------+--------+------+--------+--------+----+



In [0]:
#loading into staging data
#staging_path = "abfss://stagingdata@healthcareblob31dec.dfs.core.windows.net"
#group_data.write.format("csv").option("header","true").mode("append").option("path", staging_path).save()

In [0]:
output_container_path = "abfss://stagingdata@healthcareblob31dec.dfs.core.windows.net"
output_blob_folder = "stagingdata/"
group_data.coalesce(1).write.mode("overwrite").option("header" , "true"). format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
outputfile = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(outputfile[0].path, "%s/group_data_stage.csv"% output_container_path)

True